# Import packages and main files

In [2]:
import PyPDF2
import re, time
import requests
import pandas as pd
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()

In [3]:
url = 'https://quantinar.com/api/flower/index'
r = requests.get(url)
json_file = r.json()

In [5]:
q_master = pd.DataFrame({'id' : [json_file['data'][i]['id'] for i in range(len(json_file['data']))],
'name' : [json_file['data'][i]['name'] for i in range(len(json_file['data']))],
'team' : [json_file['data'][i]['team'] for i in range(len(json_file['data']))],
'artist' : [json_file['data'][i]['artist'] for i in range(len(json_file['data']))],
'author' : [json_file['data'][i]['author'] for i in range(len(json_file['data']))],
'published_in' : [json_file['data'][i]['published_in'] for i in range(len(json_file['data']))],
'full_link' : [json_file['data'][i]['full_link'] for i in range(len(json_file['data']))],
'pdf_url' : [json_file['data'][i]['pdf_url'] for i in range(len(json_file['data']))]
})
q_master['url_check'] = [len(i) for i in q_master['pdf_url']]
q_master = q_master.loc[q_master['url_check'] != 0, ]
q_master = q_master.reset_index(drop=True)

In [6]:
q_master_json = q_master.to_json(orient='index')
with open("q-master-json.json", "w") as outfile:
    outfile.write(q_master_json)

In [4]:
stopwords_list = {"i","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd","your","yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers","herself","it","it's","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this",'that',"that'll","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","don't","should","should've","now","d","ll","m","o","re","ve","y","ain","aren","aren't","couldn","couldn't","didn","didn't","doesn","doesn't","hadn","hadn't","hasn","hasn't","haven","haven't","isn","isn't","ma","mightn","mightn't","mustn","mustn't","needn","needn't","shan","shan't","shouldn","shouldn't","wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't"}

# Word bank Function definition
## Download function

In [5]:
def download_pdf(file_name, url):

    '''Download a PDF file with an URL'''

    # Define HTTP Headers
    headers = {"User-Agent": "Chrome/51.0.2704.103"}
    
    # Download image
    response = requests.get(url, headers=headers)
    # response = requests.get(url)
    
    # if response is OK download the PDF and store it, else write the status
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)
    
    return

## String creator

In [6]:
def create_string(file_name, url):
    
    download_pdf(file_name, url)

    '''Transform a PDF file to a list of string pages'''
    
    # opening the file
    imported_pdf = open(file_name, 'rb')
    
    # convert PDF to readable file
    transformed_pdf = PyPDF2.PdfFileReader(imported_pdf, strict=False)
    
    # get number of pages
    totalpages = transformed_pdf.numPages
    
    # read the data and store in a list
    pdf_output = [transformed_pdf.getPage(i) for i in range(totalpages)]

    # extract result
    pdf_output = [pdf_output[i].extractText() for i in range(totalpages)]
    
    return pdf_output, totalpages 

## Cleaning function

In [7]:
def cleaning(file_name, url):

    '''Initial PDF cleaning procedure'''
    
    pdf_output, totalpages = create_string(file_name, url)
    
    # # cleaning URLs
    pdf_output = [re.sub(pattern = "http[^ ]*", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning symbols
    pdf_output = [re.sub(pattern = "\\n", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "\W|\d", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "[^a-zA-Z]", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning multispaces
    pdf_output = [re.sub(pattern = "\s{2,}", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # cleaning out 1-2-worders
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    
    # # lower-casing
    pdf_output = [pdf_output[i].lower() for i in range(totalpages)]
    pdf_output = [[ps.stem(word) for word in sentence.split(" ")] for sentence in pdf_output]
    pdf_output = [' '.join(pdf_output[i]) for i in range(len(pdf_output))]
    
    return pdf_output, totalpages

## Final combined PDF creator

In [156]:
def combined_pdf_creator():
    '''Creating the final master-pdf dataframe'''

    # clean the first pdf
    pdf_output, totalpages = cleaning(str(q_master.iloc[0, 0]), q_master.iloc[0 ,7])

    # combine the pdf
    combined_pdf = [' '.join(pdf_output)]

    # iterate on above
    for i in range(1, q_master.shape[0]):
        print(i)
        t = time.process_time()
        try:
            pdf_output, totalpages = cleaning(str(q_master.iloc[i, 0]), q_master.iloc[i ,7])
            combined_pdf.append(' '.join(pdf_output))
        except:
            print('problematic file: ', str(q_master.iloc[i, 0]), q_master.iloc[i ,7])
            combined_pdf.append(' '.join(''))
        finally:
            print('time elapsed: ', (time.process_time() - t))
    return combined_pdf

# Input and search function definitions
## Search input

In [9]:
def input_sequence(input_initial): 

    '''Trimming input search terms to be used for the occurrence matrix. The output is a generalized stemmed input form ready for checking and a count of terms for the ngram_range.'''

    # splitting the phrase by pieces
    input_general = input_initial.split(' ')

    # cleaning stopwords
    input_general = [i for i in input_general if i not in stopwords_list]

    # count words
    input_general_count = len(input_general)

    # stem the words
    input_general = [ps.stem(i) for i in input_general]

    # create the additional variations of the phrase
    outer_list = []
    for i in range(0, input_general_count):
        inner_list = [input_general[j : input_general_count-i+j] for j in range(i+1)]
        outer_list.append(inner_list)

    return input_general, input_general_count, outer_list

In [10]:
def general_occurrence(input_general_count, combined_pdf): 

    '''Creation of the generalized tfidf occurance matrix based on dynamic parameters.'''

    vectorizer_general = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase=False, stop_words=stopwords_list, ngram_range=(input_general_count, input_general_count))
    X_general = vectorizer_general.fit_transform(combined_pdf)
    xx_general = pd.DataFrame(X_general.toarray(), columns = vectorizer_general.get_feature_names_out())
    return xx_general

In [179]:
def check_for_general(input_initial, input_general_count, outer_list, combined_pdf, number_of_urls):

    '''Main function.'''

    # initiating a breaker function
    breaker = 0
    # creating the occurrence matrix for max length
    xx_general = general_occurrence(input_general_count, combined_pdf)
    # creating an empty table for results
    test_output = xx_general.copy()
    test_output = test_output.iloc[:,0]*0
    # first test for full match
    print('search term: ', outer_list[0][0])
    test = ' '.join(outer_list[0][0])
    # if test passed
    if test in list(xx_general.columns):
        # create a ranked index
        ranked_indexes = xx_general[test].sort_values(ascending=False).index
        ranked_indexes = list(ranked_indexes[0:number_of_urls])
        # connect back to urls
        output_url = [q_master['id'][i] for i in ranked_indexes]
        print('search result: present\n')
        return output_url
    # if test failed drill-down
    else: 
        print('search result: not present, drill-down\n')
        for y in range(1, len(outer_list)):
            # create a new occurance matrix with new ngrams
            xx_general = general_occurrence(input_general_count-y, combined_pdf)
            for u in range(y+1):
                # drill-down phrase test
                print('search term: ', outer_list[y][u])
                test = ' '.join(outer_list[y][u])
                # if test passed
                if test in list(xx_general.columns):
                    # sum the tfidf indexes across multiple matches
                    test_output += xx_general[test]
                    print('search result: present\n')
                    # initiate the exit from the function
                    breaker = 1
                else: 
                    print('search result: not present\n')
            if breaker == 1:
                # order the indexes by highest tfidf
                ranked_indexes = test_output.sort_values(ascending=False).index
                ranked_indexes = list(ranked_indexes[0:number_of_urls])
                # return urls
                output_url = [q_master['id'][i] for i in ranked_indexes]
                return output_url

In [157]:
combined_pdf = combined_pdf_creator()

1
time elapsed:  1.2463730000000055
2
time elapsed:  0.8488540000000171
3
time elapsed:  0.6128999999999678
4
time elapsed:  0.8392120000000318
5
time elapsed:  0.8427849999999921
6
time elapsed:  1.8208640000000287
7
time elapsed:  1.0670389999999657
8
time elapsed:  0.5367709999999875
9
time elapsed:  1.9320859999999698
10
time elapsed:  0.7196099999999888
11
time elapsed:  1.4151750000000334
12
time elapsed:  1.0184229999999843
13
time elapsed:  1.2061990000000264
14
time elapsed:  0.6304279999999949
15
time elapsed:  0.2721569999999929
16
time elapsed:  0.7624889999999596
17
time elapsed:  1.6125630000000228
18
time elapsed:  1.5120539999999778
19
time elapsed:  0.5663619999999696
20
time elapsed:  0.39819800000003625
21
time elapsed:  4.035685000000001
22
time elapsed:  0.29297699999995075
23
time elapsed:  0.8151900000000296
24
time elapsed:  0.1747519999999554
25
time elapsed:  2.022788999999989
26
time elapsed:  0.8552540000000022
27
time elapsed:  1.1886740000000486
28
time el

 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M0
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M1
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M2
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M3
 impossible to decode XFormObject /M4
 impossible 

time elapsed:  2.4501670000000217
59
time elapsed:  0.29718199999996386
60
time elapsed:  0.10714100000001281
61
time elapsed:  0.23178899999999203
62
time elapsed:  0.21000100000003386
63
time elapsed:  1.1136960000000045
64
time elapsed:  0.3212270000000217
65
time elapsed:  0.9605060000000094
66
time elapsed:  1.9931909999999675
67
time elapsed:  0.2822159999999485
68
time elapsed:  0.17592500000000655
69
time elapsed:  0.5277090000000157
70
time elapsed:  0.6784900000000107
71
time elapsed:  0.5311939999999709
72
time elapsed:  0.6882640000000038
73
problematic file:  200 https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F200%2F1642686124163527012720210526+SAE+NAG+HAE+SIZ+Understanding+jumps+in+high+frequency+digital+asset+markets_course.pdf
time elapsed:  0.12702099999995653
74
time elapsed:  0.3875189999999975
75
time elapsed:  1.3995590000000107
76
time elapsed:  0.9782329999999888
77
time elapsed:  1.1899389999999812
78
time elapsed:  1.1238720000000058
79
time

In [185]:
input_initial = 'Pricing and hedging inverse BTC options'
number_of_urls = 5
input_general, input_general_count, outer_list = input_sequence(input_initial)
check_for_general(input_initial, input_general_count, outer_list, combined_pdf, number_of_urls)

search term:  ['price', 'hedg', 'invers', 'btc', 'option']
search result: present



[282, 79, 266, 239, 240]

## JSON Output script

In [303]:
input_initial = 'Pricing and hedging inverse BTC options'
number_of_urls = 5
input_general, input_general_count, outer_list = input_sequence(input_initial)
output_url = check_for_general(input_initial, input_general_count, outer_list, combined_pdf, number_of_urls)
json_output = pd.DataFrame({'id': output_url}).to_json(orient='index')

search term:  ['price', 'hedg', 'invers', 'btc', 'option']
search result: present



# Output documents

## Dictionary output

In [218]:
combined_pdf_df = pd.DataFrame({'id' : q_master['id'], "text" : combined_pdf})
combined_pdf_df
combined_pdf_json = combined_pdf_df.to_json(orient='index')
combined_pdf_json
with open("combined_pdf_json.json", "w") as outfile:
    outfile.write(combined_pdf_json)

## Check for overwriting

In [295]:
reset_index = 0
url = 'https://quantinar.com/api/flower/index'
r = requests.get(url)
json_file = r.json()

q_check = pd.DataFrame({'id' : [json_file['data'][i]['id'] for i in range(len(json_file['data']))],
'name' : [json_file['data'][i]['name'] for i in range(len(json_file['data']))],
'team' : [json_file['data'][i]['team'] for i in range(len(json_file['data']))],
'artist' : [json_file['data'][i]['artist'] for i in range(len(json_file['data']))],
'author' : [json_file['data'][i]['author'] for i in range(len(json_file['data']))],
'published_in' : [json_file['data'][i]['published_in'] for i in range(len(json_file['data']))],
'full_link' : [json_file['data'][i]['full_link'] for i in range(len(json_file['data']))],
'pdf_url' : [json_file['data'][i]['pdf_url'] for i in range(len(json_file['data']))]
})
q_check['url_check'] = [len(i) for i in q_check['pdf_url']]
q_check = q_check.loc[q_check['url_check'] != 0, ]
q_check = q_check.reset_index(drop=True)

# if triggered, then it means that the pdf downloading must happen again
if all([any(o == q_master['id']) for o in [i for i in q_check['id']]]) == False:
    reset_index == 1